# HLY 1801 Catch

- we didn't use these for the soundspeed/absorption, we used 2017 (add to methods)
- what would the impact be on backscatter, because ranges are short and frequency is low,  if the ss varied by 10% backscatter averaged would

to answer review
- yes i agree, i used water column averages from a previous year.
for text:
I used values for 2017 because this is reasonable because it is relatively representative, within the context of 4 years
- here's rationally how different it would be, and what would that mean for my abundance 

1. Age-0 gadids are largely retained change the title
2. Find every place in the paper that says arctic cod and smooth it down. 
2. 40% unidentified gadids, genetic testing is being done

In [1]:
import pandas as pd

In [51]:
hly2018 = pd.read_csv('../../data/catchData/MethotCatch.csv')
gAcod = len(hly2018[(hly2018.Latitude>=69) & (hly2018.Family=='Gadidae') & (hly2018.Species=='Arctic Cod')])
gScod = len(hly2018[(hly2018.Latitude>=69) & (hly2018.Family=='Gadidae') & (hly2018.Species=='Saffron Cod')])
gPol = len(hly2018[(hly2018.Latitude>=69) & (hly2018.Family=='Gadidae') & (hly2018.Species=='Pollock?')])
gPcod = len(hly2018[(hly2018.Latitude>=69) & (hly2018.Family=='Gadidae') & (hly2018.Species=='Pacific Cod')])
gAll = len(hly2018[(hly2018.Latitude>=69) & (hly2018.Family=='Gadidae')])
print('Total Gadids north of 69N: ',gAll/len(hly2018[(hly2018.Latitude>=69)]))
print('Total Arctic cod north of 69N: ',gAcod,',',(gAcod/len(hly2018[(hly2018.Latitude>=69)]))*100,'%')
print('Total Saffron cod north of 69N: ',gScod,',',(gScod/len(hly2018[(hly2018.Latitude>=69)]))*100,'%')
print('Total Pollock north of 69N: ',gPol,',',(gPol/len(hly2018[(hly2018.Latitude>=69)]))*100,'%')
print('Total Pacific cod north of 69N: ',gPcod,',',(gPcod/len(hly2018[(hly2018.Latitude>=69)]))*100,'%')
print('Number of tows: ',len(hly2018[(hly2018.Latitude>=69)].Tow.unique()))

Total Gadids north of 69N:  0.5110132158590308
Total Arctic cod north of 69N:  82 , 36.12334801762114 %
Total Saffron cod north of 69N:  27 , 11.894273127753303 %
Total Pollock north of 69N:  7 , 3.0837004405286343 %
Total Pacific cod north of 69N:  0 , 0.0 %
Number of tows:  9


# 2017 and 2019 CPUE
Number of fish per 100,000 cubic meters

In [56]:
eventStats = pd.read_csv('../../data/catchData/AIESMarinovichEventData.csv')
gadidRatios = pd.read_csv('../../data/catchData/eventRatios.csv')
# fix 1 missing haul which has no genetics and all pollock
gadidRatios = gadidRatios.append({'SURVEY':201901, 'EVENT_ID':404, 'acod':0, 'pollock':0, 'acodRatio':0,'pollockRatio':1},ignore_index=True)
gadidRatios= gadidRatios.drop(columns=['Unnamed: 0'])
eventStats['cpueF'] = 1/(eventStats.VOL_FILTERED/100000)
eventStats=eventStats[['SURVEY','EVENT_ID','EQ_LONGITUDE', 'EQ_LATITUDE','VOL_FILTERED','cpueF']]
os2017=pd.read_csv('../../data/catchData/OS201701MidwaterCatchSummary.csv')
os2019=pd.read_csv('../../data/catchData/OS201901MidwaterCatchSummary.csv')
catch = pd.concat([os2017,os2019])
catch = catch.rename(columns={'CLAMS_EVENT_NUMBER':'EVENT_ID'})
catchMerged = pd.merge(catch, eventStats, on=['SURVEY','EVENT_ID'], how='left')
catchMerged = catchMerged[catchMerged.SPECIES_CODE<40000]
catchMerged['cpueCatch'] = catchMerged.TOTAL_NUMBER_IN_HAUL*catchMerged.cpueF
gadids = catchMerged[((catchMerged.SPECIES_CODE== 21725)|(catchMerged.SPECIES_CODE== 21740)|(catchMerged.SPECIES_CODE== 21744))][['SHIP','SURVEY','EVENT_ID','EQ_LONGITUDE', 'EQ_LATITUDE','SPECIES_CODE','cpueCatch']]
nongadids = catchMerged[((catchMerged.SPECIES_CODE!= 21725)&(catchMerged.SPECIES_CODE!= 21740)&(catchMerged.SPECIES_CODE!= 21744))][['SHIP','SURVEY','EVENT_ID','EQ_LONGITUDE', 'EQ_LATITUDE','SPECIES_CODE','cpueCatch']]

dsList = []
for df in [gadids[gadids.SURVEY==201701],gadids[gadids.SURVEY==201901]]:
    for event in df.EVENT_ID.unique():
        for spec in [21725, 21740]:
            ds = df[(df.EVENT_ID == event)].iloc[0][['SHIP','SURVEY','EVENT_ID','EQ_LONGITUDE','EQ_LATITUDE']]
            tGadids = df[(df.EVENT_ID==event)&((df.SPECIES_CODE== 21725)|(df.SPECIES_CODE== 21740)|(df.SPECIES_CODE== 21744))].cpueCatch.sum()
            if spec == 21725:
                ds['SPECIES_CODE']= 21725
                ds['cpueCatch']=tGadids*gadidRatios[(gadidRatios.SURVEY==df.SURVEY.unique()[0]) & (gadidRatios.EVENT_ID==event)].acodRatio.values[0]
            elif spec == 21740:
                ds['SPECIES_CODE']= 21740
                ds['cpueCatch']=tGadids*gadidRatios[(gadidRatios.SURVEY==df.SURVEY.unique()[0]) & (gadidRatios.EVENT_ID==event)].pollockRatio.values[0]
            dsList.append(ds)
newGadids = pd.DataFrame(dsList)
newCatch = nongadids.append(newGadids,sort=False)

In [58]:
dsList = []
for df in [newCatch[newCatch.SURVEY==201701],newCatch[newCatch.SURVEY==201901]]:
    for event in df.EVENT_ID.unique(): 
        if df[(df.EVENT_ID == event) & (df.SPECIES_CODE==21725)].empty:
            ds = df[(df.EVENT_ID == event)].iloc[0][['SHIP','SURVEY','EVENT_ID','EQ_LONGITUDE','EQ_LATITUDE']]
            ds['propAcod']=0
        else:
            ds = df[(df.EVENT_ID == event)].iloc[0][['SHIP','SURVEY','EVENT_ID','EQ_LONGITUDE','EQ_LATITUDE']]
            ds['propAcod']=df[(df.EVENT_ID == event) & (df.SPECIES_CODE==21725)].cpueCatch.values[0]/df[df.EVENT_ID == event].cpueCatch.sum()
        dsList.append(ds)
propAcod = pd.DataFrame(dsList)

In [70]:
print('2017 # tows:',len(newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)].EVENT_ID.unique()))
print('2019 # tows:',len(newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)].EVENT_ID.unique()))

2017 # tows: 28
2019 # tows: 31


In [60]:
newCatch = newCatch[(newCatch.SPECIES_CODE != 20203) & (newCatch.SPECIES_CODE != 20204)]
print('2017 - Genetics: ',\
newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)& (newCatch.SPECIES_CODE ==21725)].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum(),
newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)& ((newCatch.SPECIES_CODE ==21744) | (newCatch.SPECIES_CODE ==21740))].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum(),
newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)& (newCatch.SPECIES_CODE ==21735)].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum(),
newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)& (newCatch.SPECIES_CODE ==21720)].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum())

print('gadid sum', sum([newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)& (newCatch.SPECIES_CODE ==21725)].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum(),
newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)& ((newCatch.SPECIES_CODE ==21744) | (newCatch.SPECIES_CODE ==21740))].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum(),
newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)& (newCatch.SPECIES_CODE ==21735)].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum(),
newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)& (newCatch.SPECIES_CODE ==21720)].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum()]))


print('2019 - Genetics: ',\
newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)& (newCatch.SPECIES_CODE ==21725)].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum(),
newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)& ((newCatch.SPECIES_CODE ==21744) | (newCatch.SPECIES_CODE ==21740))].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum(),
newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)& (newCatch.SPECIES_CODE ==21735)].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum(),
newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)& (newCatch.SPECIES_CODE ==21720)].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum())

print('gadid sum', sum([newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)& (newCatch.SPECIES_CODE ==21725)].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum(),
newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)& ((newCatch.SPECIES_CODE ==21744) | (newCatch.SPECIES_CODE ==21740))].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum(),
newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)& (newCatch.SPECIES_CODE ==21735)].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum(),
newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)& (newCatch.SPECIES_CODE ==21720)].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum()]))


2017 - Genetics:  0.8548619686636874 0.09787805651639392 9.739440484650274e-05 6.684863699539684e-05
gadid sum 0.9529042682219231
2019 - Genetics:  0.4554744663143731 0.4008699468327388 0.000735917623591869 0.0006127037208199144
gadid sum 0.8576930344915236


In [41]:
newCatch = nongadids.append(gadids,sort=False)
newCatch = newCatch[(newCatch.SPECIES_CODE != 20203) & (newCatch.SPECIES_CODE != 20204)]
print('2017 - Uncorrected: ',\
newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)& (newCatch.SPECIES_CODE ==21725)].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum(),
newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)& ((newCatch.SPECIES_CODE ==21744) | (newCatch.SPECIES_CODE ==21740))].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum(),
newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)& (newCatch.SPECIES_CODE ==21735)].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum(),
newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)& (newCatch.SPECIES_CODE ==21720)].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum())

print('gadid sum', sum([newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)& (newCatch.SPECIES_CODE ==21725)].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum(),
newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)& ((newCatch.SPECIES_CODE ==21744) | (newCatch.SPECIES_CODE ==21740))].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum(),
newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)& (newCatch.SPECIES_CODE ==21735)].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum(),
newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)& (newCatch.SPECIES_CODE ==21720)].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201701)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum()]))

print('2019 - Uncorrected: ',\
newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)& (newCatch.SPECIES_CODE ==21725)].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum(),
newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)& ((newCatch.SPECIES_CODE ==21744) | (newCatch.SPECIES_CODE ==21740))].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum(),
newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)& (newCatch.SPECIES_CODE ==21735)].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum(),
newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)& (newCatch.SPECIES_CODE ==21720)].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum())

print('gadid sum',sum([newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)& (newCatch.SPECIES_CODE ==21725)].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum(),
newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)& ((newCatch.SPECIES_CODE ==21744) | (newCatch.SPECIES_CODE ==21740))].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum(),
newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)& (newCatch.SPECIES_CODE ==21735)].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum(),
newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)& (newCatch.SPECIES_CODE ==21720)].cpueCatch.sum()/newCatch[(newCatch.SURVEY==201901)& (newCatch.EQ_LATITUDE >69)].cpueCatch.sum()]))

2017 - Uncorrected:  0.9527385465981593 1.478581921972874e-06 9.739440484650273e-05 6.684863699539683e-05
gadid sum 0.9529042682219231
2019 - Uncorrected:  0.7074461198623203 0.14889829328479173 0.0007359176235918691 0.0006127037208199145
gadid sum 0.8576930344915238


### In a table:

|Survey|# Tows|A.Cod|Pollock|S.Cod|PCod|% Gadids|
|---|---|---|---|---|---|---|
|2017|28 |95.2 |<1 |<1 |<1 |95.3 |
|2017 - Genetics|28 |85.4 |9.7 |<1 |<1 |95.3 |
|2018| 9 | 36.1|3.1 | 11.9| 0| 51.1|
|2019|31 | 70.7|14.9|<1 |<1 |85.7 |
|2019 - Genetics|31 |45.5 |40.0 |<1 |<1 |85.7|